In [ ]:
# 获取环境变量
from google.colab import userdata
import os
ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

In [ ]:
# 确保文件存在
import pandas as pd
import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/movies_analysis'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/drive/MyDrive/movies_analysis/movie_reviews.xlsx
/content/drive/MyDrive/movies_analysis/movie_description.xlsx
/content/drive/MyDrive/movies_analysis/movies_analysis_score.xlsx
/content/drive/MyDrive/movies_analysis/my_movie_analysis_score.ipynb


In [ ]:
# 看看文件
df_raw = pd.read_excel("/content/drive/MyDrive/movies_analysis/movie_reviews.xlsx")
df_raw.head()

,review_id,review
0,1,...this is not that.\n\nWhile it was certainly...
1,2,While I am loathe to criticise a fairly origin...
2,3,"The camera ""inside"" the gladiator helmet is ju..."
3,4,Very pleased with this show so far. Seems to p...
4,5,"If you enjoy the series, do NOT read the books..."


In [ ]:
# 对每个列进行空值检查,强迫症
df_raw.isnull().sum()

review_id    0
review       0
dtype: int64

In [ ]:
# 安装langchain库
!pip install langchain-anthropic
# 更新一下版本
!pip install -qU langchain-anthropic

In [ ]:
# 引入模型
from langchain_anthropic import AnthropicLLM
from langchain_core.prompts import PromptTemplate

# warn忽略,问就是老模型就是这个版本
model = AnthropicLLM(model='claude-2.1')

# 测试的电影评论
this_review = """While I am loathe to criticise a fairly original story and something quite different from the norm, this film was definitely not for me. I'm a big fan of Stone, Ruffalo and Defoe so it pains me even more to be negative but I did not like this at all and feel tricked by the high review scores, to the point where I feel the need to leave a low score to balance it out a bit.

I was left thinking suddenly I no longer understand film for this to have scored so high.

I'm not sure why this is deemed a comedy, it is much more a horror in the conventional sense. Frequently grotesque and music designed to make you want to hit the mute button. I tried to like the style and cinematography but I just found the whole story so unpleasant as to be completely distracting.

The one thing I did find funny was the first frame saying 'CONTAINS TOBACCO REFERENCES', which is then proceeded by the opening shot of a suicide. What a bizarre world we are living in."""
# 随手写一个prompt模板
prompt = PromptTemplate.from_template("""Classify the sentiment of the movie review into 10 levels, ranging from 1 to 10, with 10 being the most positive and 1 being the most negative. Rate the current movie based on the review:
{review}

just return the score with nothing else""")

/usr/local/lib/python3.10/dist-packages/langchain_anthropic/llms.py:176: UserWarning: This Anthropic LLM is deprecated. Please use `from langchain_community.chat_models import ChatAnthropic` instead
  warnings.warn(


In [ ]:
from langchain_core.output_parsers import StrOutputParser
# 定义一个chain
my_chain =  prompt | model | StrOutputParser()
# 测试执行
my_chain.invoke({"review": this_review})

'\n3'

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_anthropic import ChatAnthropic
# 写个函数
def getScore(review):
  # 换个版本.tmd
  llm = ChatAnthropic(temperature=0,model_name="claude-3-opus-20240229")
  # prompt模板
  prompt = PromptTemplate.from_template("""Classify the sentiment of the movie review into 10 levels, ranging from 1 to 10, with 10 being the most positive and 1 being the most negative. Rate the current movie based on the review:
  {review}

  just return the score with nothing else""")
  # 定义一个chain
  my_chain =  prompt | llm | StrOutputParser()
  return my_chain.invoke({"review": review})

In [ ]:
# 逐行读取review列
for index, row in df_raw.iterrows():
    review = row['review']
    review_id = str(row['review_id'])
    print(review+"\n\n\n")
    score = getScore(review)
    cleaned_score = score.replace("\n", "")
    print("###########"+review_id+"###########"+review+"###########"+cleaned_score+"###########")
    break

...this is not that.

While it was certainly groundbreaking and a huge hit at the time, in retrospect great parts of it was rather cheasy and cheapish.

The 2024 version feels a great deal more modern and budgeted bigger, it's rougher, it's dirtier, the action scenes are crafted much better, the camera work is better, and so far the script seems better than the original series, with better dialogue allowing the characters more complexity and debth, which is especially important to American and European audiences, when depicting a very "foreign" culture.

I should mention this is based off the first 2 episodes, so we are yet to see large scale battle scenes, but looking at the teaser clips, they look exellent, so I am hopefull.

It's a show that - even after 2 episodes - makes you annoyed it is a limited series, by nature of James Clavell having written only one novel on feudal Japan.. but I actually predict this will be such a big hit, that FX will attempt to have scripts written for e

/usr/local/lib/python3.10/dist-packages/langchain_anthropic/llms.py:176: UserWarning: This Anthropic LLM is deprecated. Please use `from langchain_community.chat_models import ChatAnthropic` instead
  warnings.warn(


###########1###########...this is not that.

While it was certainly groundbreaking and a huge hit at the time, in retrospect great parts of it was rather cheasy and cheapish.

The 2024 version feels a great deal more modern and budgeted bigger, it's rougher, it's dirtier, the action scenes are crafted much better, the camera work is better, and so far the script seems better than the original series, with better dialogue allowing the characters more complexity and debth, which is especially important to American and European audiences, when depicting a very "foreign" culture.

I should mention this is based off the first 2 episodes, so we are yet to see large scale battle scenes, but looking at the teaser clips, they look exellent, so I am hopefull.

It's a show that - even after 2 episodes - makes you annoyed it is a limited series, by nature of James Clavell having written only one novel on feudal Japan.. but I actually predict this will be such a big hit, that FX will attempt to hav

In [ ]:
# 测试输出
output_file_path = "/content/drive/MyDrive/movies_analysis/movies_analysis_score.xlsx"

data = {
    'review_id': [1, 2, 3, 4, 5],
    'review': ['Good product', 'Bad experience', 'Great service', 'Average', 'Excellent'],
    'sentiment': [4.5, 2.0, 5.0, 3.0, 5.0]
}

df = pd.DataFrame(data)

df.to_excel(output_file_path, index=False)
# 看看文件
df_raw2 = pd.read_excel(output_file_path)
df_raw2.head()

,review_id,review,sentiment
0,1,Good product,4.5
1,2,Bad experience,2.0
2,3,Great service,5.0
3,4,Average,3.0
4,5,Excellent,5.0


In [ ]:
import time
# 数据产出
review_ids=[]
reviews=[]
sentiments=[]
for index, row in df_raw.iterrows():
    review = row['review']
    review_id = str(row['review_id'])
    score = getScore(review)
    # 执行快了,说什么超过限额
    time.sleep(0.5)
    cleaned_score = score.replace("\n", "")
    print("###########"+review_id+"###########"+cleaned_score+"###########")
    review_ids.append(review_id)
    reviews.append(review)
    sentiments.append(cleaned_score)


output_file_path = "/content/drive/MyDrive/movies_analysis/movies_analysis_score.xlsx"

data = {
    'review_id': review_ids,
    'review': reviews,
    'sentiment': sentiments
}

df = pd.DataFrame(data)

df.to_excel(output_file_path, index=False)
# 看看文件
df_raw2 = pd.read_excel(output_file_path)
df_raw2.head()

###########1###########8###########
###########2###########3###########
###########3###########2###########
###########4###########9###########
###########5###########3###########
###########6###########8###########
###########7###########3###########
###########8###########9###########
###########9###########9###########
###########10###########9###########
###########11###########9###########
###########12###########3###########
###########13###########9###########
###########14###########7###########
###########15###########2###########
###########16###########10###########
###########17###########10###########
###########18###########8###########
###########19###########9###########
###########20###########4###########
###########21###########2###########
###########22###########4###########
###########23###########9###########
###########24###########9###########
###########25###########7###########
###########26###########1###########
###########27###########10###########
#######

,review_id,review,sentiment
0,1,...this is not that.\n\nWhile it was certainly...,8
1,2,While I am loathe to criticise a fairly origin...,3
2,3,"The camera ""inside"" the gladiator helmet is ju...",2
3,4,Very pleased with this show so far. Seems to p...,9
4,5,"If you enjoy the series, do NOT read the books...",3
